In [1]:
!pip install pennylane



     |████████████████████████████████| 481kB 8.4MB/s 


## Qubit Rotation


The task at hand is to optimize two rotation gates in order to flip a single qubit from state 
$|0⟩$ to state $|1⟩$

### The Quantum Circuit

Breaking this down step-by-step, we first start with a qubit in the ground state |0⟩ = $[1 , 0]^T$
, and rotate it around the x-axis by applying the gate

$R_x(ϕ_1)$= $e^{−iϕ_1σ_x/2}$

and then around the y-axis via the gate

$R_y(ϕ_2)$=$e^{−iϕ_2σ_y/2}$

After these operations the qubit is now in the state

$|ψ⟩$=
$R_y(ϕ_2)R_x(ϕ_1)|0⟩$
.
Finally, we measure the expectation value 
$⟨
ψ
∣
σ
_z
∣
ψ
⟩$
 of the Pauli-Z operator
Using the above to calculate the exact expectation value, we find that

$⟨
ψ
∣
σ
_z
∣
ψ
⟩
=
⟨
0
∣
R
_x
(
ϕ
_1
)
†
R
_y
(
ϕ
_2
)
†
σ
_z
R
_y
(
ϕ
_2
)
R
_x
(
ϕ
_1
)
∣
0
⟩
=
cos
(
ϕ
_1
)
cos
(
ϕ
_2
)$
.
Depending on the circuit parameters 
$ϕ
_1$
 and 
$ϕ
_2$
, the output expectation lies between 
1
 (if 
$|
ψ
⟩
=
|
0
⟩
) $and 
−
1
 (if 
 $
|
ψ
⟩
=
|
1
⟩
).$



In [2]:
import pennylane as qml
from pennylane import numpy as np

It is important to load Numpy from the pennylane library , not from the standard numpy

## Creating A Device

Any computational object that can apply quantum operations, and return an measurement value is called a quantum device.

In [3]:
dev1 = qml.device("default.qubit", wires = 1)

Default Qubit is just the Pure State Qubit simulator and wires is the number of subsytems to be initialized

Hence Single Qubit , so wires = 1

## Constructing the QNode

QNodes are an abstract encapsulation of a quantum function, described by a quantum circuit. QNodes are bound to a particular quantum device, which is used to evaluate expectation and variance values of this circuit.



In [5]:
def circuit(params):
  qml.RX(params[0], wires=0)
  qml.RX(params[1], wires=0)
  return qml.expval(qml.PauliZ(0))


Once we have written the Quantum Function, we can convert it into a QNode, running on dev1 device

In [6]:
@qml.qnode(dev1)
def circuit(params):
  qml.RX(params[0], wires=0)
  qml.RX(params[1], wires=0)
  return qml.expval(qml.PauliZ(0))


Thus, Now the circuit() function is a QNode, which will run on dev1


In [7]:
print(circuit([0.54, 0.12]))

0.7899922314973652


## Calculating Quantum Gradient

The gradient of the function circuit, encapsulated within the QNode, can be evaluated by utilizing the same quantum device (dev1) that we used to evaluate the function itself.



In [8]:
dcircuit = qml.grad(circuit, argnum= 0)

The function grad() itself, returns a function, representing the derivative of the QNode with respect to the argument in argnum.

In this case, the function circuit takes one argument (params) , so we specify argnum =0 .

Because the argument has 2 elements, the returned gradient is two dimensional

In [11]:
print(dcircuit([0.54, 0.12]))

[array(-0.61311685), array(-0.61311685)]


Quantum Circuit Functions, are a restricted subset of Python functions, we could have defined the circuit as follows

In [12]:
@qml.qnode(dev1)
def circuit2(phi1, phi2):
  qml.RX(phi1, wires=0)
  qml.RY(phi2, wires=0)
  return qml.expval(qml.PauliZ(0))

But when we calculate the gradient for the given function, the argnum , would be slightly different.

Here argnum = 0, will only give the gradient wrt phi1 and argnum = 1 will give the gradient wrt phi2

In [14]:
dcircuit2 = qml.grad(circuit2, argnum=[0,1])
print(dcircuit([0.54,0.12]))

[array(-0.61311685), array(-0.61311685)]


## Optimization

Let us make use of built in optimizers to rotate the state initially in 0 to the 1 state

Which is equivalen of Pauli-Z measuring the expectation value of -1.

The optimization function will find the best weights for the parameters phi1 and phi2


In [15]:
def cost(x):
  return circuit(x)

In [16]:
# chose some initial value for it
init_params = np.array([0.011, 0.012])
print(cost(init_params))

0.999735511659836


Hence for the initial parameters, the cost function is close to 1

We will iterate it 100 times

In [19]:
# initialize
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# number of steps
steps = 100

params = init_params

for i in range(steps):
  params = opt.step(cost,params)


  if (i+1) % 5 == 0:
    print("Cost after step {:5d} : {: .7f}".format(i+1, cost(params)))


print("Optimized rotation angles : {}".format(params))

Cost after step     5 :  0.9081672
Cost after step    10 : -0.9970866
Cost after step    15 : -1.0000000
Cost after step    20 : -1.0000000
Cost after step    25 : -1.0000000
Cost after step    30 : -1.0000000
Cost after step    35 : -1.0000000
Cost after step    40 : -1.0000000
Cost after step    45 : -1.0000000
Cost after step    50 : -1.0000000
Cost after step    55 : -1.0000000
Cost after step    60 : -1.0000000
Cost after step    65 : -1.0000000
Cost after step    70 : -1.0000000
Cost after step    75 : -1.0000000
Cost after step    80 : -1.0000000
Cost after step    85 : -1.0000000
Cost after step    90 : -1.0000000
Cost after step    95 : -1.0000000
Cost after step   100 : -1.0000000
Optimized rotation angles : [1.57029633 1.57129633]


We can see that the optimization converges after approximately 15 steps.